# TOKENS Y EMBEDDINGS

En esta lección veremos qué son los tokens y los embeddings, dos conceptos clave al momento de representar el texto que ingresa a un LLM.

Además, tendremos una primera aproximación a un LLM y veremos en acción estos dos conceptos a través de un ejemplo práctico. Y adicionalmente, al final de la lección hablaremos de la **ventana de contexto**, un concepto también relacionado con los tokens.

## 1. El problema con el texto

![](https://drive.google.com/uc?export=view&id=1XkzcdFcmbeIUA9FS2t52daZ6pcOc8Jnz)


En esencia, el texto es un tipo de dato **no estructurado** y es necesario darle algún tipo de estructura al momento de llevarlo a un LLM.

## 2. La solución: tokens + embeddings

![](https://drive.google.com/uc?export=view&id=1DZxXh612aWcScIJU6TozUzjOjmkfKDMn)

En un LLM el texto de entrada debe pasar inicialmente por dos fases de pre-procesamiento:

1. **La "tokenización"**: donde se subdivide el texto en palabras o sub-palabras y el resultado se representa con un índice numérico
2. **La generación de "embeddings" de entrada** donde se genera una **representación vectorial inicial** de cada índice numérico

Veamos en detalle cada una de estas etapas:

### 2.1. La "tokenización"

![](https://drive.google.com/uc?export=view&id=1CeGyRyfnazzIU8A5n9o9cwuunrc04qNA)

Es la primera fase que permite representar de forma estructurada el texto. Las etapas de esta tokenización son:

1. Subdividir el texto en palabras o sub-palabras (esto último es más eficiente que únicamente usar palabras). El resultado son **los tokens**
2. Posteriormente, cada token se representa con un identificador numérico único
3. Es importante tener en cuenta que cada LLM tiene **tokens especiales** para indicar, por ejemplo, inicio y finalización de frases, padding, palabras desconocidas, etc.

Teniendo esto claro, veamos en acción cómo funcionan esta tokenización:

Comencemos instalando la librería Transformers de Hugging Face, que nos permitirá acceder a una gran variedad de LLMs:

In [4]:
!pip install transformers==4.48.3

Para usar un LLM generalmente debemos importar dos elementos:

- Un "tokenizer" para generar los tokens del texto de entrada
- Y el modelo

> Es importante tener en cuenta que el "tokenizer" y el "modelo" **están interconectados**, es decir que ambos son creados de manera simultánea al momento de entrenar el modelo

In [5]:
from transformers import AutoModel, AutoTokenizer

# Cargar modelo y tokenizador
modelo_name = "microsoft/deberta-v3-xsmall"
tokenizer = AutoTokenizer.from_pretrained(modelo_name)
modelo = AutoModel.from_pretrained(modelo_name)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Comencemos viendo cómo crear la tokenización:

In [6]:
# Texto de entrada al LLM
frase = "banco de madera, banco donde hay dinero"

# Tokenización del texto
tokens = tokenizer(frase, return_tensors="pt")

# Obtención de los ids correspondientes a cada token
token_ids = tokens.input_ids[0]

# Comparar número de palabras y de tokens
print("Número de palabras: ", len(frase.split()))
print("Número de tokens: ", len(token_ids))

Número de palabras:  7
Número de tokens:  13


Como se mencionó anteriormente, los tokens pueden ser **palabras o sub-palabras** así que el número de tokens generado siempre será mayor que el número de palabras.

> En general esta relación número de tokens / número de palabras puede oscilar entre 1.2 y 2 aproximadamente (todo depende del LLM que estemos usando)

Veamos la frase anterior ya tokenizada:

In [7]:
token_ids

tensor([     1,   4797,   1902,    718,    412,   3608,    261,   4797,   1902,
         66346,  12530, 107725,      2])

In [8]:
# Mostrar cada token
for id in token_ids:
    print(tokenizer.decode(id))

[CLS]
ban
co
de
made
ra
,
ban
co
donde
hay
dinero
[SEP]


¡Y acá vemos precisamente cómo algunos tokens contienen palabras completas y otros sub-palabras!

Además podemos observar dos cosas:

1. Los signos de puntuación también se tokenizan
2. Y el tokenizer incluye automáticamente los tokens especiales cuando sea necesario

Veamos la codificación numérica de cada token:

In [9]:
# Mostrar cada token y su id correspondiente
for id in tokens.input_ids[0]:
    print(tokenizer.decode(id), "----->", id.item())

[CLS] -----> 1
ban -----> 4797
co -----> 1902
de -----> 718
made -----> 412
ra -----> 3608
, -----> 261
ban -----> 4797
co -----> 1902
donde -----> 66346
hay -----> 12530
dinero -----> 107725
[SEP] -----> 2


En esencia esta codificación se hace a través de un diccionario token -> token_id.

El conjunto total de tokens en este diccionario es lo que llamamos **el vocabulario** y el tamaño de este vocabulario será simplemente el número total de tokens diferentes que puede llegar a analizar el modelo:

In [10]:
# Mostrar el tamaño del vocabulario
print(len(tokenizer))

128001


### 2.2. Los "embeddings"

La representación de cada token a través de "token_ids" no es aún adecuada para que el modelo la pueda interpretar.

Así que todo LLM tiene un primer bloque de entrada, conocido como **input embedding** (embedding de entrada), que convierte cada "token_id" en un vector:

![](https://drive.google.com/uc?export=view&id=1GlZTQgZ92AG59xoPhWcI8CJmMs_qfU1x)


Los "embeddings" cumplen con estos propósitos:

1. Estandarizan cada token: sin importar el número de caracteres del token, todos son representados con un vector **de tamaño fijo**
2. Los "embeddings" generados tienen valores entre -1 y 1 generalmente, lo que los hace adecuados para entrenar y luego generar predicciones con el LLM
3. Este primer bloque de generación de "embeddings" realiza la codificación de cada token e **intenta preservar su significado o contenido semántico**: palabras **aisladas** con significados similares tendrán "embeddings" similares (más adelante estos "embeddings" serán refinados usando el mecanismo atencional de los LLMs)

Veamos cómo el modelo genera estos "embeddings" a la entrada. Comencemos viendo en detalle el modelo que cargamos hace unos momentos:

In [11]:
modelo

DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(128100, 384, padding_idx=0)
    (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0-11): 12 x DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=384, out_features=384, bias=True)
            (key_proj): Linear(in_features=384, out_features=384, bias=True)
            (value_proj): Linear(in_features=384, out_features=384, bias=True)
            (pos_dropout): Dropout(p=0.1, inplace=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-07, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

En el caso anterior podemos ver absolutamente todas las capas que conforman este LLM. Sin embargo, en este caso únicamente nos interesa la capa de entrada:


`(word_embeddings): Embedding(128100, 384, padding_idx=0)`

Comprendamos el significado de esta porción:

- `(word_embeddings)`: el nombre dado a esta capa por los creadores del modelo
- `Embedding`: se trata precisamente de una capa que genera embeddings.
- `128100`: el tamaño del vocabulario
- `384`: el tamaño de cada "embedding" generado por este bloque
- `padding_idx=0`: indica que los token_ids que tengan valores de cero serán tratados como el token especial de padding (relleno)

En esencia una capa de embedding es una simple matriz donde cada fila es un "embedding" y cada posición de fila corresponde a un token en particular.

Esta matriz se entrena junto con el modelo y por eso es esencial que el tokenizer y el modelo estén sincronizados durante la carga.

Así que ahora tomaremos los "token_ids" y se los presentaremos únicamente a esta capa de "embedding":




In [13]:
token_ids

tensor([     1,   4797,   1902,    718,    412,   3608,    261,   4797,   1902,
         66346,  12530, 107725,      2])

In [14]:
# Procesar la secuencia con el modelo y extraer únicamente los word_embeddings
import torch

# Extract word embeddings
with torch.no_grad():
    embeddings = modelo.embeddings.word_embeddings(token_ids)



Veamos el tamaño de estos "embeddings":

In [15]:
embeddings.shape

torch.Size([13, 384])

El tamaño es:

- 1: pues hemos presentado sólo una frase al modelo
- 15: pues la frase contiene 15 tokens
- 384: pues cada token es representado con un "embedding" (vector) de 384 elementos

Veamos uno de estos embeddings:

In [18]:
print(f"Token: {tokenizer.decode(token_ids[6])} --> {token_ids[6]}")
print(f"Embedding correspondiente:")
embeddings[6]

Token: , --> 261
Embedding correspondiente:


tensor([ 4.3213e-02, -5.9814e-03,  7.9102e-02,  1.0431e-01,  8.7891e-02,
         6.3843e-02,  7.6111e-02,  1.0071e-01,  1.0614e-01,  5.4626e-03,
        -1.0437e-02,  4.8065e-02,  9.0027e-04,  4.6814e-02, -2.6855e-03,
         2.0630e-02,  1.1719e-02,  9.1309e-02,  9.3689e-03, -6.2988e-02,
         1.2299e-01,  9.4177e-02,  1.0095e-01, -5.4382e-02,  2.0752e-03,
         5.6030e-02,  3.4790e-02,  6.7627e-02,  1.1749e-02,  5.4688e-02,
        -1.4404e-02,  2.9846e-02, -9.0637e-03,  2.6291e-02,  4.0405e-02,
        -1.6602e-02, -9.2773e-03,  7.9346e-02,  6.1340e-02,  4.4525e-02,
         3.4790e-02,  6.7505e-02,  1.0913e-01,  3.2135e-02,  2.8839e-02,
         2.7740e-02,  6.8115e-02,  8.0078e-02,  2.2430e-02, -3.9612e-02,
        -1.2781e-01,  6.5613e-02, -8.8257e-02,  2.6520e-02, -7.8467e-01,
         2.8259e-02,  1.2451e-01,  4.7150e-02,  2.3438e-02,  1.2207e-02,
         8.0444e-02,  3.7781e-02,  4.8950e-02,  4.6875e-02,  6.8909e-02,
         4.5715e-02,  9.3018e-02,  9.8877e-03,  4.5

Además, si tenemos tokens idénticos (por ejemplo los tokens 2 y 8 son "ban") sus "embeddings" serán exactamente los mismos:

In [20]:
# Tokens 2 y 8
print(tokenizer.decode(token_ids[1]))
print(tokenizer.decode(token_ids[7]))

ban
ban


In [22]:
# Mostrar embeddings correspondientes
print(embeddings[1])
print(embeddings[7])

tensor([-8.8501e-04, -4.3555e-01,  1.2079e-01, -1.9043e-01,  3.7415e-02,
        -7.7332e-02, -2.0947e-01,  2.5659e-01, -3.6963e-01, -3.3936e-01,
         3.6548e-01,  1.5125e-01,  9.5703e-02,  6.8848e-02, -5.3320e-01,
        -2.3438e-01,  1.6382e-01, -2.8369e-01, -3.5620e-01, -4.9194e-02,
        -7.3120e-02,  7.1716e-02, -2.2852e-01, -3.4424e-01, -3.1982e-01,
        -2.4048e-02, -9.9182e-02,  1.4185e-01, -6.0486e-02, -3.3105e-01,
        -3.0304e-02, -5.8624e-02,  6.1584e-02, -3.9600e-01,  1.1877e-01,
         1.3123e-02, -1.7676e-01, -9.0881e-02,  3.0005e-01,  3.3783e-02,
         7.7515e-02,  1.7102e-01,  8.9539e-02, -1.8884e-01,  3.1445e-01,
         9.9243e-02,  1.1719e-01,  4.5557e-01, -1.4893e-02, -1.7078e-01,
         6.7993e-02, -2.6001e-01,  1.2732e-01, -1.6205e-02, -1.4160e-02,
        -1.5503e-01, -1.6724e-02, -2.1008e-01, -3.9734e-02, -2.9443e-01,
         1.8799e-01,  1.2323e-01, -1.1145e-01,  3.5474e-01, -1.4087e-01,
        -2.8833e-01, -7.8125e-02,  1.0730e-01, -1.6

Y verifiquemos que estos "embeddings" son idénticos:

In [23]:
# Mostrar resta de estos embeddings
print(embeddings[1] - embeddings[7])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

## 3. La ventana de contexto

> Es simplemente el número máximo de tokens que podrá procesar el LLM

Así que en esencia, la ventana de contexto define el tamaño máximo que podrá tener la secuencia para que el modelo la pueda procesar.

Este tamaño lo podemos encontrar accediendo a los detalles de la configuración del modelo:

In [24]:
from transformers import AutoConfig

# Configuración del modelo
config = AutoConfig.from_pretrained(modelo_name)

# Print max sequence length
print(f"Ventana de contexto: {config.max_position_embeddings}")


Ventana de contexto: 512
